In [1]:
# Importing the required libraries
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error

import pickle

import warnings
warnings.filterwarnings('ignore')

### Question: 01

In [2]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

# Number of columns in dataset
len(df.columns)

19

### Question: 02

In [3]:
# Calculating the duration in minutes
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df['duration'].std()

42.59435124195458

### Question: 03

In [4]:
# Records after drop
number_records_before_drop = df.shape[0]

df = df[(df.duration >= 1) & (df.duration <= 60)]
number_records_after_drop = df.shape[0]

percentage_left = (number_records_after_drop / number_records_before_drop) * 100
print(f"{percentage_left:.2f}")

98.12


### Question: 04

In [5]:
# Cast the IDs to strings
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

# Convert DataFrame to a list of dictionaries
train_dicts = df[categorical].to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Display feature names and feature matrix
feature_names = dv.get_feature_names_out()

# Number of columns
len(feature_names)

515

### Question: 05

In [6]:
# Training linear regression model
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.649261932106969

### Question: 06

In [7]:
# Define a function
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
# Reading data
df_train = read_dataframe('./data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2023-02.parquet')

In [9]:
# Preparing training data
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [10]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

: 

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)